## Building a Credit Scoring Model for Buy-Now-Pay-Later Services: A Comprehensive Analysis of Credit Risk Classification

## Overview
Credit scoring models are vital tools used by financial institutions to assess the creditworthiness of potential borrowers. As part of risk management, these models predict the likelihood of a borrower defaulting on a loan, which allows institutions to mitigate potential losses. Traditionally, credit scoring models rely on statistical methods to analyze historical data, identifying patterns and relationships between borrower behavior and loan outcomes.

In this challenge, we aim to build a comprehensive credit scoring model for Bati Bank, a leading financial institution collaborating with an eCommerce platform. The goal is to enhance their buy-now-pay-later service, allowing customers to purchase items on credit, based on their predicted creditworthiness. By developing a reliable and robust credit scoring model, Bati Bank can make informed lending decisions while minimizing the risk of defaults. This project involves data exploration, feature engineering, model development, and real-time deployment of the model via an API to assess credit risk and optimize loan terms.

## Objectives
The primary objectives of this project are as follows:

1. Define Credit Risk Proxy: Establish a proxy variable to categorize users as high-risk (bad) or low-risk (good) based on their likelihood of default.

2. Feature Engineering: Select relevant features from the data and engineer new ones that are strong predictors of default risk. This includes creating aggregate and extracted features, encoding categorical variables, handling missing data, and normalizing numerical features.

3. Develop a Credit Risk Model: Build and train machine learning models that assign risk probabilities to new customers based on historical transaction data.

4. Credit Scoring: Use the model's probability estimates to create a credit score for each customer, facilitating quick and accurate creditworthiness assessments.

5. Loan Optimization: Predict the optimal loan amount and duration for new customers, considering their risk profile to ensure sustainable lending practices.

6. Model Deployment: Deploy the trained credit scoring model through an API, enabling real-time credit scoring and decision-making. The API will accept customer transaction data and return predictions on credit risk and loan recommendations.

### Import Library

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load dataset

In [2]:
df = pd.read_csv('../data/data.csv')
df

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1000.0,1000,2019-02-13T09:54:09Z,2,0
95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2019-02-13T09:54:25Z,2,0
95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2019-02-13T09:54:35Z,2,0
95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,ChannelId_3,3000.0,3000,2019-02-13T10:01:10Z,2,0


## 3. Feature Engineering

### 3.1 Create Aggregate Features

In [3]:
# Creating aggregate features
aggregate_features = df.groupby('CustomerId').agg(
    Total_Transaction_Amount=('Amount', 'sum'),
    Average_Transaction_Amount=('Amount', 'mean'),
    Transaction_Count=('TransactionId', 'count'),
    Std_Deviation_Transaction_Amount=('Amount', 'std')
).reset_index()

# Display the aggregate features
aggregate_features


,CustomerId,Total_Transaction_Amount,Average_Transaction_Amount,Transaction_Count,Std_Deviation_Transaction_Amount
0,CustomerId_1,-10000.0,-10000.000000,1,NaN
1,CustomerId_10,-10000.0,-10000.000000,1,NaN
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333
3,CustomerId_1002,4225.0,384.090909,11,560.498966
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146
...,...,...,...,...,...
3737,CustomerId_992,20000.0,3333.333333,6,6088.240030
3738,CustomerId_993,20000.0,4000.000000,5,6745.368782
3739,CustomerId_994,543873.0,5384.881188,101,14800.656784
3740,CustomerId_996,139000.0,8176.470588,17,4433.329648


### 3.2 Extract Features

In [4]:
# Convert TransactionStartTime to datetime
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])

# Extracting features
df['Transaction_Hour'] = df['TransactionStartTime'].dt.hour
df['Transaction_Day'] = df['TransactionStartTime'].dt.day
df['Transaction_Month'] = df['TransactionStartTime'].dt.month
df['Transaction_Year'] = df['TransactionStartTime'].dt.year

# Display the updated DataFrame with new features
df[['TransactionId', 'CustomerId', 'Transaction_Hour', 'Transaction_Day', 'Transaction_Month', 'Transaction_Year']]


,TransactionId,CustomerId,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Year
0,TransactionId_76871,CustomerId_4406,2,15,11,2018
1,TransactionId_73770,CustomerId_4406,2,15,11,2018
2,TransactionId_26203,CustomerId_4683,2,15,11,2018
3,TransactionId_380,CustomerId_988,3,15,11,2018
4,TransactionId_28195,CustomerId_988,3,15,11,2018
...,...,...,...,...,...,...
95657,TransactionId_89881,CustomerId_3078,9,13,2,2019
95658,TransactionId_91597,CustomerId_3874,9,13,2,2019
95659,TransactionId_82501,CustomerId_3874,9,13,2,2019
95660,TransactionId_136354,CustomerId_1709,10,13,2,2019


In [5]:
# Display the final DataFrame with aggregate features and extracted features
final_df = df.merge(aggregate_features, on='CustomerId', how='left')

# Reorder the columns to place 'FraudResult' at the end
column_order = [col for col in final_df.columns if col != 'FraudResult'] + ['FraudResult']
final_df = final_df[column_order]

# Display the final DataFrame with aggregate features and extracted features
final_df



,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,PricingStrategy,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Year,Total_Transaction_Amount,Average_Transaction_Amount,Transaction_Count,Std_Deviation_Transaction_Amount,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,...,2,2,15,11,2018,109921.75,923.712185,119,3042.294251,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,...,2,2,15,11,2018,109921.75,923.712185,119,3042.294251,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,...,2,2,15,11,2018,1000.00,500.000000,2,0.000000,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,...,2,3,15,11,2018,228727.20,6019.136842,38,17169.241610,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,...,2,3,15,11,2018,228727.20,6019.136842,38,17169.241610,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,...,2,9,13,2,2019,2438140.00,4255.043630,573,22554.029939,0
95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,...,2,9,13,2,2019,58499.60,1360.455814,43,2274.756582,0
95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,...,2,9,13,2,2019,58499.60,1360.455814,43,2274.756582,0
95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,...,2,10,13,2,2019,851985.00,1625.925573,524,3207.920536,0


In [6]:
final_df.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'Transaction_Hour',
       'Transaction_Day', 'Transaction_Month', 'Transaction_Year',
       'Total_Transaction_Amount', 'Average_Transaction_Amount',
       'Transaction_Count', 'Std_Deviation_Transaction_Amount', 'FraudResult'],
      dtype='object')

### 3.3 Encodding

#### One-Hot Encoding

In [7]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# One-Hot Encoding
one_hot_columns = ['ProductCategory', 'ChannelId', 'CurrencyCode']
df_one_hot = pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

# Display the DataFrame after One-Hot Encoding
df_one_hot.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProviderId,ProductId,Amount,Value,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,256,ProviderId_6,ProductId_10,1000.0,1000,...,False,False,False,False,False,False,False,False,True,False
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,256,ProviderId_4,ProductId_6,-20.0,20,...,True,False,False,False,False,False,False,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,256,ProviderId_6,ProductId_1,500.0,500,...,False,False,False,False,False,False,False,False,True,False
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,256,ProviderId_1,ProductId_21,20000.0,21800,...,False,False,False,False,False,False,True,False,True,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,256,ProviderId_4,ProductId_6,-644.0,644,...,True,False,False,False,False,False,False,True,False,False


#### Label Encoding

In [8]:
# Label Encoding
label_columns = ['ProviderId', 'PricingStrategy', 'CountryCode']
label_encoder = LabelEncoder()

for col in label_columns:
    df_one_hot[col] = label_encoder.fit_transform(df_one_hot[col])

# Display the DataFrame after Label Encoding
df_one_hot.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProviderId,ProductId,Amount,Value,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,5,ProductId_10,1000.0,1000,...,False,False,False,False,False,False,False,False,True,False
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,3,ProductId_6,-20.0,20,...,True,False,False,False,False,False,False,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,5,ProductId_1,500.0,500,...,False,False,False,False,False,False,False,False,True,False
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,0,ProductId_21,20000.0,21800,...,False,False,False,False,False,False,True,False,True,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,3,ProductId_6,-644.0,644,...,True,False,False,False,False,False,False,True,False,False


In [9]:
# Display the final DataFrame
df_one_hot.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CountryCode,ProviderId,ProductId,Amount,Value,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,0,5,ProductId_10,1000.0,1000,...,False,False,False,False,False,False,False,False,True,False
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,0,3,ProductId_6,-20.0,20,...,True,False,False,False,False,False,False,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,0,5,ProductId_1,500.0,500,...,False,False,False,False,False,False,False,False,True,False
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,0,0,ProductId_21,20000.0,21800,...,False,False,False,False,False,False,True,False,True,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,0,3,ProductId_6,-644.0,644,...,True,False,False,False,False,False,False,True,False,False


### 3.4 Data Cleaning

#### Identify Missing Values

In [10]:
# Check for missing values
missing_values = final_df.isnull().sum()
print("Missing values in each column:\n", missing_values[missing_values > 0])


Missing values in each column:
 Std_Deviation_Transaction_Amount    712
dtype: int64


#### Handling Missing Values

In [11]:
# Imputation
for column in final_df.select_dtypes(include=['float64', 'int64']).columns:
    median_value = final_df[column].median()
    final_df[column].fillna(median_value, inplace=True)

for column in final_df.select_dtypes(include=['object']).columns:
    mode_value = final_df[column].mode()[0]
    final_df[column].fillna(mode_value, inplace=True)


C:\Users\getac\AppData\Local\Temp\ipykernel_2500\798469771.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[column].fillna(median_value, inplace=True)
C:\Users\getac\AppData\Local\Temp\ipykernel_2500\798469771.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [12]:
final_df.isnull().sum()

TransactionId                       0
BatchId                             0
AccountId                           0
SubscriptionId                      0
CustomerId                          0
CurrencyCode                        0
CountryCode                         0
ProviderId                          0
ProductId                           0
ProductCategory                     0
ChannelId                           0
Amount                              0
Value                               0
TransactionStartTime                0
PricingStrategy                     0
Transaction_Hour                    0
Transaction_Day                     0
Transaction_Month                   0
Transaction_Year                    0
Total_Transaction_Amount            0
Average_Transaction_Amount          0
Transaction_Count                   0
Std_Deviation_Transaction_Amount    0
FraudResult                         0
dtype: int64

### 3.5 Normalize/Standardize Numerical Features

In [13]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Separate numerical columns for normalization and standardization
numerical_columns = final_df.select_dtypes(include=['float64', 'int64']).columns

# Normalization
min_max_scaler = MinMaxScaler()
final_df[numerical_columns] = min_max_scaler.fit_transform(final_df[numerical_columns])

# Display the DataFrame after scaling
print("DataFrame after normalization:\n")
final_df.head()


DataFrame after normalization:



,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,PricingStrategy,Transaction_Hour,Transaction_Day,Transaction_Month,Transaction_Year,Total_Transaction_Amount,Average_Transaction_Amount,Transaction_Count,Std_Deviation_Transaction_Amount,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,airtime,...,0.5,2,15,11,2018,0.557522,0.047184,0.028851,0.000919,0.0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,financial_services,...,0.5,2,15,11,2018,0.557522,0.047184,0.028851,0.000919,0.0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,airtime,...,0.5,2,15,11,2018,0.556944,0.047137,0.000244,0.000000,0.0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,utility_bill,...,0.5,3,15,11,2018,0.558153,0.047749,0.009046,0.005187,0.0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,financial_services,...,0.5,3,15,11,2018,0.558153,0.047749,0.009046,0.005187,0.0


#### Save the feature engineering Data

In [14]:
final_df.to_csv('../data/feature_engineerd_data.csv')